# 8章 ディープラーニング

ディープラーニングは，層を深くしたディープなニューラルネットワークである．層を重ねるだけでディープなネットワークを作れるが，課題もある．

## 8.1 ネットワークをより深く

これまで学んだ技術を集約して，ディープなネットワークを作り，MNISTデータセットの手書き数字認識に挑む．

### 8.1.1 よりディープなネットワークへ

#### 手書き数字認識を行うディープなCNN

- 3×3の小さなフィルターによる畳み込み層
- 活性化関数はReLU
- 全結合層の後にDropoutレイヤを使用
- Adamによる最適化
- 重みの初期値として「Heの初期値」を使用

#### ソースコード
- ネットワークを実装したコード ch08/deep_convnet.py
- 訓練用のコード ch08/train_deepnet.py
- 学習済みのパラメータ ch08/deep_convnet_params.pkl
- パラメータを読み込む機能 deep_convnet.py

### 8.1.2 さらに認識精度を高めるには

手書き数字という比較的単純な問題に対しては，ネットワークの表現力をそこまで高める必要がないが，一般物体認識は，問題が複雑になるため，層を深くすることが認識精度の向上に大いに貢献する．

アンサンブル学習，学習係数の減衰，DataAugmentation（データ拡張）などは認識精度の向上に貢献している．

Data Augmentation：入力画像（訓練画像）をアルゴリズムによって，人工的に拡張する．具体的にあh，入力画像に対して，回転や縦横方向の移動などの微小な変化を与え，画像枚数を増やすことを行う．データセットの枚数が限られている場合には有効．

### 8.1.3 層を深くすることのモチベーション

#### ネットワークのパラメータ数を小さくできる
パラメータ数の差は層が深くなるにつれて大きくなるので利点が増す．
#### 学習の効率が上がる
ネットワークを深くすることで学習すべき問題を階層的に分解することができる．→各層が学習すべき問題は，より単純な問題として取り組むことができる．

## 8.2 ディープラーニングの小歴史

### 8.2.1 ImageNet

ImageNetは100万枚を超える画像のデータセット．

### 8.2.2 VGG

VGG：畳み込み層とプーリング層から構成される基本的なCNN．

#### ポイント
3×3の小さなフィルターによる畳み込み層を連続して行っている．畳み込み層を2回から4回連続し，プーリング層でサイズを半分にするという処理を繰り返し行う．最後に全結合層を経由して結果を出力する．

### 8.2.3 GoogLeNet

ネットワークが縦方向の深さだけではなく，横方向にも深さ（広がり）を持っている点が特徴である．

### 8.2.4 ResNet

スキップ構造を導入することで，層を深くすることに比例して，性能を向上させることができるようになった．

スキップ構造：入力データの畳み込み層をまたいでスキップして-出力に合算する構造

スキップ構造は入力データをそのまま流すだけなので，逆伝播時も，上流からの勾配をそのまま下流へ流す．スキップ構造によって，勾配が小さくなったり（大きくなりすぎたり）する心配がなく，前層のレイヤに「意味のある勾配」が伝わっていくことが期待できる．→層を深くすることで勾配が小さくなる勾配消失問題はこのスキップ構造で軽減することができる．

#### 転移学習
学習済みの重み（の一部）を別のニューラルネットワークにコピーして，再学習を行う．例えば，VGGと同じ構成のネットワークを用意し，学習済みの重みを初期値とし，新しいデータセットを対象に，再学習(fine tuning)を行う．

転移学習は，手元にあるデータセットが少ない場合において，特に有効な手法である．

## 8.3 ディープラーニングの高速化

ビッグデータとネットワークの大規模化により，ディープラーニングでは大量の演算を行う必要がある．これまではCPUを使っていたが，最近のディープラーニングのフレームワークの多くはGPUをサポートしており，大量の演算を高速に処理可能．

さらに複数のGPUや複数台のマシンでの分散学習にも対応し始めている．

### 8.3.1 取り組むべき問題

#### ディープラーニングでは畳み込み層で行われる計算をいかに高速に効率よく行うかというい点がディープラーニングでの課題になる．

### 8.3.2 GPUによる高速化

GPUは元々，グラフィックのための専用のボードとして利用されていた．しかし最近では，グラフィック処理だけでなく，汎用的な数値計算にもGPUは利用できる．

#### GPUコンピューティング
GPUによって汎用的な数値計算を行うということ．
- GPU:大量にある並列的な演算と得意とする
- CPU:連続的で複雑な計算を得意とする

#### GPUを提供する2社
- NVIDIA:（CUDA：GPUコンピューティング向けの統合開発環境）
- AMD

畳み込み層で行う演算はim2colによって，大きな行列の積に変換することができる．im2col方式の実装は，GPUにとって都合の良い実装方式である．

### 8.3.3 分散学習

#### 分散学習
1回の学習に要する時間をできるだけ小さくするためにディープラーニングの学習をスケールアウトさせようという考え．

現在のディープラーニングのフレームワークでは，複数GPUや複数マシンによる分散学習をサポートしたものがいくつか現れてきている．

#### 代表的な分散学習ツール
- TensorFlow(Google)
- Microsoft(CNTK)

### 8.3.4 演算制度のビット削減

ディープラーニングの高速化においては，計算量に加えて，メモリ容量やバス帯域などがボトルネックなりえる．
- メモリ容量：大量の重みパラメータや中間データをメモリに収めることを考慮する必要がある．
- バス帯域：GPU（CPU）のバスを流れるデータが増加してある制限を超えるとそこがボトルネックになる．

→ネットワークを流れるデータのビット数はできるだけ小さくすることが望まれる．

#### 半精度浮動小数点数
ディープラーニングにおいては，半精度浮動小数点数でも問題なく学習できることがわかっている．

## 8.4 ディープラーニングの実用例

ディープラーニングは物体認識だけではなく，画像や音声，自然言語など，分野は違えど，多くの問題に対して優れた性能を発揮する．

### 8.4.1 物体検出

#### 画像中から物体の位置の特定を含めてクラス分類を行う問題．

### 8.4.2 セグメンテーション

#### 画像に対してピクセルレベルでクラス分類を行う問題

ニューラルネットワークによって，セグメンテーションを行う最も単純な方法は，すべてのピクセルを対象として，ピクセルごとに推論処理を行うこと．（時間かかる）

#### FCN(Fully Convolutional Network)
1回のforward処理ですべてのピクセルに対してクラス分類を行う．

### 8.4.3 画像キャプション生成

#### コンピュータビジョンと自然言語を融合した研究
画像を与えると，その画像を説明する文章（画像キャプション）を自動で生成する．

- NIC：ディープなCNNと自然言語を扱うためのRNNから構成
- RNN：再帰的なつながりを持つネットワーク．自然言語や時系列データなどの連続性のあるデータに対して用いられる．
- マルチモーダル処理：画像と自然言語といったように複数の種類の情報を組み合わせて処理すること．

## 8.5 ディープラーニングの未来

### 8.5.1 画像スタイル変換

#### 2つの画像を入力し，新しい画像を生成する．
2つの画像のうち一つは，「コンテンツ画像」，もう一つは「スタイル画像」と呼び，2枚の画像を入力すると新しい画像が生成される．

### 8.5.2 画像生成

#### DOGAN: GeneratorとDiscriminatorと呼ばれる2つのニューラルネットワークを利用し，両者を競わせるように学習させることで，成長していく．

### 8.5.3 自動運転

### 8.5.4 Deep Q-Network（強化学習）

#### 強化学習：人が試行錯誤を経て学ぶようにコンピュータが試行錯誤の過程から，自律的に学習させようという分野．

## 8.6 まとめ

- 多くの問題では，ネットワークを深くすることで，性能の向上が期待できる．
- ILSVRCと呼ばれる画像認識のコンペティションの最近の動向は，ディープラーニングによる手法が上位を独占し，使われるネットワークもディープ化している．
- 有名なネットワークには，VGG，GoogLeNet，ResNetがある．
- GPUや分散学習，びっとせいど
-
-